In [1]:
import os
import itertools

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
from tqdm import tqdm

from datagen import *
from network import *
from benchmarks import *

from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler

In [3]:
def expand_grid(data_dict):
    """Create a dataframe from every combination of given values."""
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

## Two Strategies

- Strategy 1 (S1): point estimator (avg wts) + CI (draw from avg wts) 
- Strategy 2 (S2): point estimator (single wt) + CI (draw from single wts)
<!-- - Strategy 3 (S3): point estimator (avg wt) + CI (draw from single wts) -->

In [4]:
def allocation_test(mdist_obs,mdist_array):
    return (np.sum(mdist_array>=mdist_obs)+1)/(mdist_array.shape[0]+1)

In [5]:
save_folder_root = './save/0525qrer_sp/'

In [6]:
def parallel_unit(i,r,batch_size,
                  lr,pa,
                  num_nodes,num_iters,
                  num_init_iters,
                  x_lambda,
                  wt_lambda,
                  kernel_params,val_metric,
                  patience,random_state,
                  data_path):
  
    print('------------- Data:',i,'------------- ')

    # load the data
    data_full_path = data_path + 'd' + str(i) + '.npy'
    dat = np.load(data_full_path,allow_pickle=True)

    # unzip the data
    x = dat.item()['x']
    z = dat.item()['z']
    y1 = dat.item()['y1']
    y2 = dat.item()['y2']
    y3 = dat.item()['y3']
        
    nt = int(z.sum())
    x = x[:nt*(1+r)]
    z = z[:nt*(1+r)]
    y1 = y1[:nt*(1+r)]
    y2 = y2[:nt*(1+r)]
    y3 = y3[:nt*(1+r)]
    
    sc = StandardScaler()
    x = sc.fit_transform(x)
    
    # only use the mean difference loss
    save_folder = save_folder_root+scenario+'/'+'r='+str(r)+'/pa='+str(pa)+'/'+str(i)+'/'

    if not os.path.exists(save_folder):
        print('Creat the folder.')
        os.makedirs(save_folder)

    if not os.path.exists(save_folder+'final_checkpoint.pt'):
        print('Train the model from scratch.')

        estimator = QRWG(lr=lr,
                          batch_size=batch_size,
                          patience=patience,
                          num_iters=num_iters,
                          num_init_iters=num_init_iters,
                          pa=pa,
                          x_lambda=x_lambda,
                          wt_lambda=wt_lambda,
                          num_nodes=num_nodes,
                          val_metric=val_metric,
                          save_folder=save_folder,
                          kernel_params=kernel_params,
                          verbose=False,
                          random_state=random_state)

        # train the model from scratch
        estimator.fit(x,z)

    else:
        print('Skip! The model has been trained.')
        estimator = QRWG(lr=lr,
                          batch_size=batch_size,
                          patience=patience,
                          num_iters=num_iters,
                          num_init_iters=1,
                          pa=pa,
                          x_lambda=x_lambda,
                          wt_lambda=wt_lambda,
                          num_nodes=num_nodes,
                          val_metric=val_metric,
                          save_folder=save_folder,
                          kernel_params=kernel_params,
                          verbose=False,
                          random_state=random_state)
        estimator.w = z
        estimator.nwts = int(estimator.w.shape[0])
        estimator.nt = int(z.sum())
        estimator.nc = int((1-z).sum())
        estimator._init_network()
        estimator.netG.load_state_dict(torch.load(save_folder+'final_checkpoint.pt'))
    
    np.random.seed(i)
    torch.manual_seed(i)
    
    z_rer = ReR(pa,torch.Tensor(x),np.sum(z))[0].numpy()
    wts_mat_net = estimator.predict(1000).numpy()
    
    if not os.path.exists(save_folder+'zmat.npy'):
        z_rer_mat = np.array([ReR(pa,torch.Tensor(x),np.sum(z))[0].numpy() for i in range(1000)])
        np.save(save_folder+'zmat.npy',z_rer_mat)
    else:
        z_rer_mat = np.load(save_folder+'zmat.npy')
        
    if not os.path.exists(save_folder+'mdist.npy'):
        mdist_array = np.array([ReR(pa,torch.Tensor(x),np.sum(z))[1].item() for i in range(1000)])
        np.save(save_folder+'mdist.npy',mdist_array)
    else:
        mdist_array = np.load(save_folder+'mdist.npy')
    
    #z_rer_mat = np.array([ReR(pa,torch.Tensor(x),np.sum(z))[0].numpy() for i in range(1000)])
    #mdist_array = np.array([ReR(pa,torch.Tensor(x),np.sum(z))[1].item() for i in range(1000)])
    
    
    if not os.path.exists(save_folder+'qrwg_est.csv'):
        # compare different strategies
        # strategy 1:
        wts = wts_mat_net[0]

        est1_s1, lb1_s1, ub1_s1 = sp_infer(y1,z,wts)
        est2_s1, lb2_s1, ub2_s1 = sp_infer(y2,z,wts)
        est3_s1, lb3_s1, ub3_s1 = sp_infer(y3,z,wts)
        
        df_est_s1 = pd.DataFrame({
          'tauhat': [est1_s1,est2_s1,est3_s1],
          "95CI_lb": [lb1_s1,lb2_s1,lb3_s1],
          "95CI_ub": [ub1_s1,ub2_s1,ub3_s1],
          'type': 'S1'
        })

        # strategy 2:
        avg_wts = wts_mat_net.mean(axis=0)

        est1_s2, lb1_s2, ub1_s2 = sp_infer(y1,z,avg_wts)
        est2_s2, lb2_s2, ub2_s2 = sp_infer(y2,z,avg_wts)
        est3_s2, lb3_s2, ub3_s2 = sp_infer(y3,z,avg_wts)

        df_est_s2 = pd.DataFrame({
          'tauhat': [est1_s2,est2_s2,est3_s2],
          "95CI_lb": [lb1_s2,lb2_s2,lb3_s2],
          "95CI_ub": [ub1_s2,ub2_s2,ub3_s2],
          'type': 'S2'
        })

        # strategy 3:
        try:
            wts = wts_mat_net[np.cumsum(test_array>0.15)==1][0]
        except:
            wts_mat_net_tmp = estimator.predict(1000).numpy()
            test_array_tmp = np.array([allocation_test(maha_dist(x,z,wts_mat_net_tmp[i]).item(),mdist_array) for i in range(1000)])
            while np.mean(test_array_tmp>0.15)==0:
                wts_mat_net_tmp = estimator.predict(1000).numpy()
                test_array_tmp = np.array([allocation_test(maha_dist(x,z,wts_mat_net_tmp[i]).item(),mdist_array) for i in range(1000)])
            wts = wts_mat_net_tmp[np.cumsum(test_array_tmp>0.15)==1][0]
        
        est1_s3, lb1_s3, ub1_s3 = sp_infer(y1,z,wts)
        est2_s3, lb2_s3, ub2_s3 = sp_infer(y2,z,wts)
        est3_s3, lb3_s3, ub3_s3 = sp_infer(y3,z,wts)

        df_est_s3 = pd.DataFrame({
          'tauhat': [est1_s3,est2_s3,est3_s3],
          "95CI_lb": [lb1_s3,lb2_s3,lb3_s3],
          "95CI_ub": [ub1_s3,ub2_s3,ub3_s3],
          'type': 'S3'
        })
        
        df_est = pd.concat([df_est_s1,df_est_s2,df_est_s3],axis=0)
        df_est.to_csv(save_folder+"qrwg_est.csv",index=False)
        
    else:
        print('Skip! QRWG has been considered')

    return pd.read_csv(save_folder+"qrwg_est.csv").values

In [7]:
n_kernel = 40
n_data = 200
tau = 1

scenarios = ['scenario1','scenario2','scenario3']
rs = [1,2]

result1_df = []
result2_df = []
result3_df = []

In [8]:
for scenario in scenarios:
    for r in rs:
        
        print('----------------',scenario,'r =',r,'----------------')
        
        data_path = './save/simu_data/'+scenario+'/'

        net_params = {
         'r':[r],
         'data_path': [data_path],
            'batch_size': [512],
         'lr': [0.001],
         'pa': [0.1],
         'val_metric': ['KS'],
         'num_nodes': [512],
         'num_iters': [5000],
         'num_init_iters': [500],
         'patience': [15],
         'x_lambda': [1],
         'wt_lambda': [1],
         'kernel_params': [{'kernel':'rbf',
                         'gamma':10,
                         'degree':2,
                         'c':0}],
         'random_state': [0]}
        
        param_df = expand_grid(net_params)

        for i_param in range(param_df.shape[0]):
            kwargs = dict(param_df.iloc[i_param,:])
            print('----------------- [%d/%d] -----------------\n'%(i_param+1,param_df.shape[0]))
            results_all = Parallel(n_jobs=n_kernel)(delayed(parallel_unit)(i=i,**kwargs) for i in tqdm(range(n_data)))

            # qrwg 
            dat_array = np.array([results_all[i] for i in range(n_data)])
            bias = dat_array[:,:,0].mean(axis=0)-tau
            rmse = np.sqrt(np.mean((dat_array[:,:,0]-tau)**2,axis=0).astype(float))
            covarage = ((dat_array[:,:,1]<=tau)*(dat_array[:,:,2]>=tau)).mean(axis=0)
            width = (dat_array[:,:,2] - dat_array[:,:,1]).mean(axis=0)

            result_dict = {'Bias':bias,
                      'RMSE':rmse,
                      'CI Covarage':covarage,
                      'CI Width':width,
                      'pa':param_df.iloc[i_param,4],
                      'Method':['QReR-'+dat_array[0,i,3] for i in range(dat_array.shape[1])],
                      'Outcome': 3*['Linear','Nonlinear1','Nonlinear2'],
                      'r':r,
                      'Scenario':scenario}

            result_df = pd.DataFrame(result_dict)
            result1_df.append(result_df.iloc[[0,3,6],:])
            result2_df.append(result_df.iloc[[1,4,7],:])
            result3_df.append(result_df.iloc[[2,5,8],:])

  0%|          | 0/200 [00:00<?, ?it/s]

---------------- scenario1 r = 1 ----------------
----------------- [1/1] -----------------



  0%|          | 0/200 [00:00<?, ?it/s]

---------------- scenario1 r = 2 ----------------
----------------- [1/1] -----------------



  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

---------------- scenario3 r = 1 ----------------
----------------- [1/1] -----------------



  0%|          | 0/200 [00:00<?, ?it/s]

---------------- scenario3 r = 2 ----------------
----------------- [1/1] -----------------



100%|██████████| 200/200 [57:35<00:00, 17.28s/it]


In [9]:
# net_params = {
#  'r':[1],
#  'data_path': ['./save/simu_data/'+'scenario1'+'/'],
#     'batch_size': [512],
#  'lr': [0.001],
#  'pa': [0.1],
#  'val_metric': ['KS'],
#  'num_nodes': [512],
#  'num_iters': [5000],
#  'num_init_iters': [500],
#  'patience': [10],
#  'x_lambda': [0.1],
#  'wt_lambda': [1],
#  'kernel_params': [{'kernel':'rbf',
#                  'gamma':5,
#                  'degree':2,
#                  'c':0}],
#  'random_state': [0]}

# param_df = expand_grid(net_params)

In [10]:
# kwargs = dict(param_df.iloc[0,:])

In [11]:
# # load the data
# r = 1
# i = 1
# data_path = './save/simu_data/'+'scenario1'+'/'
# data_full_path = data_path + 'd' + str(i) + '.npy'
# dat = np.load(data_full_path,allow_pickle=True)

# # unzip the data
# x = dat.item()['x']
# z = dat.item()['z']
# y1 = dat.item()['y1']
# y2 = dat.item()['y2']
# y3 = dat.item()['y3']

# nt = int(z.sum())
# x = x[:nt*(1+r)]
# z = z[:nt*(1+r)]
# y1 = y1[:nt*(1+r)]
# y2 = y2[:nt*(1+r)]
# y3 = y3[:nt*(1+r)]

# sc = StandardScaler()
# x = sc.fit_transform(x)

In [12]:
# kwargs

In [13]:
# estimator = QRWG(lr=kwargs['lr'],
#                   batch_size=kwargs['batch_size'],
#                   patience=kwargs['patience'],
#                   num_iters=kwargs['num_iters'],
#                   num_init_iters=kwargs['num_init_iters'],
#                   pa=kwargs['pa'],
#                   x_lambda=kwargs['x_lambda'],
#                   wt_lambda=kwargs['wt_lambda'],
#                   num_nodes=kwargs['num_nodes'],
#                   val_metric=kwargs['val_metric'],
#                   save_folder='./save/',
#                   kernel_params=kwargs['kernel_params'],
#                   verbose=False,
#                   random_state=0)

# # train the model from scratch
# estimator.fit(x,z)

In [14]:
pd.concat(result1_df).set_index(['r','Scenario','Outcome'])

Bias      RMSE  CI Covarage  CI Width   pa   Method
r Scenario  Outcome                                                         
1 scenario1 Linear   0.000886  0.362929        1.000  2.433858  0.1  QReR-S1
            Linear  -0.024032  0.117627        1.000  2.205995  0.1  QReR-S2
            Linear   0.024394  0.278415        1.000  2.409702  0.1  QReR-S3
2 scenario1 Linear  -0.020876  0.313310        1.000  2.221624  0.1  QReR-S1
            Linear   -0.01416  0.102374        1.000  1.982931  0.1  QReR-S2
            Linear   -0.00764  0.268750        1.000  2.187933  0.1  QReR-S3
1 scenario2 Linear   0.006204  0.431706        0.995   2.97281  0.1  QReR-S1
            Linear  -0.028998  0.110695        1.000  2.676393  0.1  QReR-S2
            Linear    0.04492  0.336930        1.000  2.936937  0.1  QReR-S3
2 scenario2 Linear  -0.038879  0.354199        1.000  2.792378  0.1  QReR-S1
            Linear   -0.01797  0.096632        1.000   2.48259  0.1  QReR-S2
            Linear   0.027136  0.310703        1.000  2.725294  0.1  QReR-S3
1 scenario3 Linear  -0.026048  0.465427        1.000  3.111185  0.1  QReR-S1
            Linear  -0.036471  0.119738        1.000  2.808645  0.1  QReR-S2
            Linear   0.048944  0.372756        1.000  3.084218  0.1  QReR-S3
2 scenario3 Linear  -0.044795  0.357505        1.000  2.876144  0.1  QReR-S1
            Linear  -0.014528  0.098140        1.000  2.590228  0.1  QReR-S2
            Linear  -0.045671  0.296884        1.000  2.867652  0.1  QReR-S3

In [15]:
pd.concat(result2_df).set_index(['r','Scenario','Outcome'])

Bias      RMSE  CI Covarage  CI Width   pa  \
r Scenario  Outcome                                                      
1 scenario1 Nonlinear1 -0.014162  0.515248          1.0  3.501385  0.1   
            Nonlinear1 -0.035941  0.192624          1.0  3.171703  0.1   
            Nonlinear1   0.02707  0.407236          1.0  3.468821  0.1   
2 scenario1 Nonlinear1 -0.086054  0.438670          1.0  3.147452  0.1   
            Nonlinear1 -0.076695  0.180156          1.0  2.811564  0.1   
            Nonlinear1  -0.08059  0.395728          1.0  3.092428  0.1   
1 scenario2 Nonlinear1 -0.054878  0.622510          1.0  4.191412  0.1   
            Nonlinear1 -0.118426  0.238152          1.0    3.7782  0.1   
            Nonlinear1 -0.009568  0.521141          1.0  4.143908  0.1   
2 scenario2 Nonlinear1 -0.191611  0.552807          1.0  3.874796  0.1   
            Nonlinear1 -0.141882  0.235179          1.0  3.451441  0.1   
            Nonlinear1 -0.079106  0.455338          1.0  3.791876  0.1   
1 scenario3 Nonlinear1 -0.130431  0.667645          1.0  4.387123  0.1   
            Nonlinear1 -0.136731  0.254859          1.0  3.955234  0.1   
            Nonlinear1  -0.02318  0.505352          1.0  4.341341  0.1   
2 scenario3 Nonlinear1 -0.181618  0.552194          1.0  3.984012  0.1   
            Nonlinear1 -0.150681  0.240618          1.0  3.592942  0.1   
            Nonlinear1 -0.192379  0.470000          1.0  3.978324  0.1   

                         Method  
r Scenario  Outcome              
1 scenario1 Nonlinear1  QReR-S1  
            Nonlinear1  QReR-S2  
            Nonlinear1  QReR-S3  
2 scenario1 Nonlinear1  QReR-S1  
            Nonlinear1  QReR-S2  
            Nonlinear1  QReR-S3  
1 scenario2 Nonlinear1  QReR-S1  
            Nonlinear1  QReR-S2  
            Nonlinear1  QReR-S3  
2 scenario2 Nonlinear1  QReR-S1  
            Nonlinear1  QReR-S2  
            Nonlinear1  QReR-S3  
1 scenario3 Nonlinear1  QReR-S1  
            Nonlinear1  QReR-S2  
            Nonlinear1  QReR-S3  
2 scenario3 Nonlinear1  QReR-S1  
            Nonlinear1  QReR-S2  
            Nonlinear1  QReR-S3

In [16]:
pd.concat(result3_df).set_index(['r','Scenario','Outcome'])

Bias      RMSE  CI Covarage   CI Width   pa  \
r Scenario  Outcome                                                       
1 scenario1 Nonlinear2  0.052929  2.336547        0.950   8.186365  0.1   
            Nonlinear2  0.016747  2.028463        0.960   7.357344  0.1   
            Nonlinear2  0.062248  2.251676        0.955   8.047511  0.1   
2 scenario1 Nonlinear2 -0.267236  1.744876        0.965   6.633194  0.1   
            Nonlinear2 -0.265661  1.616926        0.960   6.131855  0.1   
            Nonlinear2 -0.270153  1.788382        0.955   6.674616  0.1   
1 scenario2 Nonlinear2  4.974949  7.380769        0.810  18.048606  0.1   
            Nonlinear2  4.865528  7.012475        0.795  16.658327  0.1   
            Nonlinear2  4.939963  7.429543        0.795  17.992443  0.1   
2 scenario2 Nonlinear2  4.647283  7.189381        0.850  17.068099  0.1   
            Nonlinear2  4.792842  6.770991        0.775   15.44743  0.1   
            Nonlinear2  4.702933  6.840538        0.785   16.75432  0.1   
1 scenario3 Nonlinear2 -5.922794  7.109124        0.695  16.326879  0.1   
            Nonlinear2 -5.793506  6.804635        0.660   14.88713  0.1   
            Nonlinear2 -5.971246  7.161557        0.700  16.067744  0.1   
2 scenario3 Nonlinear2 -4.507679  6.150922        0.750  14.744702  0.1   
            Nonlinear2 -4.706129  5.893102        0.690  13.366878  0.1   
            Nonlinear2 -4.802474  6.273839        0.695   14.59291  0.1   

                         Method  
r Scenario  Outcome              
1 scenario1 Nonlinear2  QReR-S1  
            Nonlinear2  QReR-S2  
            Nonlinear2  QReR-S3  
2 scenario1 Nonlinear2  QReR-S1  
            Nonlinear2  QReR-S2  
            Nonlinear2  QReR-S3  
1 scenario2 Nonlinear2  QReR-S1  
            Nonlinear2  QReR-S2  
            Nonlinear2  QReR-S3  
2 scenario2 Nonlinear2  QReR-S1  
            Nonlinear2  QReR-S2  
            Nonlinear2  QReR-S3  
1 scenario3 Nonlinear2  QReR-S1  
            Nonlinear2  QReR-S2  
            Nonlinear2  QReR-S3  
2 scenario3 Nonlinear2  QReR-S1  
            Nonlinear2  QReR-S2  
            Nonlinear2  QReR-S3

In [17]:
result_df = pd.concat(result1_df+result2_df+result3_df).set_index(['r','Scenario','Outcome'])

In [18]:
result_df.to_csv(save_folder_root+'qrwg_sp.csv')

In [19]:
result_df

Bias      RMSE  CI Covarage   CI Width   pa  \
r Scenario  Outcome                                                       
1 scenario1 Linear      0.000886  0.362929        1.000   2.433858  0.1   
            Linear     -0.024032  0.117627        1.000   2.205995  0.1   
            Linear      0.024394  0.278415        1.000   2.409702  0.1   
2 scenario1 Linear     -0.020876  0.313310        1.000   2.221624  0.1   
            Linear      -0.01416  0.102374        1.000   1.982931  0.1   
            Linear      -0.00764  0.268750        1.000   2.187933  0.1   
1 scenario2 Linear      0.006204  0.431706        0.995    2.97281  0.1   
            Linear     -0.028998  0.110695        1.000   2.676393  0.1   
            Linear       0.04492  0.336930        1.000   2.936937  0.1   
2 scenario2 Linear     -0.038879  0.354199        1.000   2.792378  0.1   
            Linear      -0.01797  0.096632        1.000    2.48259  0.1   
            Linear      0.027136  0.310703        1.000   2.725294  0.1   
1 scenario3 Linear     -0.026048  0.465427        1.000   3.111185  0.1   
            Linear     -0.036471  0.119738        1.000   2.808645  0.1   
            Linear      0.048944  0.372756        1.000   3.084218  0.1   
2 scenario3 Linear     -0.044795  0.357505        1.000   2.876144  0.1   
            Linear     -0.014528  0.098140        1.000   2.590228  0.1   
            Linear     -0.045671  0.296884        1.000   2.867652  0.1   
1 scenario1 Nonlinear1 -0.014162  0.515248        1.000   3.501385  0.1   
            Nonlinear1 -0.035941  0.192624        1.000   3.171703  0.1   
            Nonlinear1   0.02707  0.407236        1.000   3.468821  0.1   
2 scenario1 Nonlinear1 -0.086054  0.438670        1.000   3.147452  0.1   
            Nonlinear1 -0.076695  0.180156        1.000   2.811564  0.1   
            Nonlinear1  -0.08059  0.395728        1.000   3.092428  0.1   
1 scenario2 Nonlinear1 -0.054878  0.622510        1.000   4.191412  0.1   
            Nonlinear1 -0.118426  0.238152        1.000     3.7782  0.1   
            Nonlinear1 -0.009568  0.521141        1.000   4.143908  0.1   
2 scenario2 Nonlinear1 -0.191611  0.552807        1.000   3.874796  0.1   
            Nonlinear1 -0.141882  0.235179        1.000   3.451441  0.1   
            Nonlinear1 -0.079106  0.455338        1.000   3.791876  0.1   
1 scenario3 Nonlinear1 -0.130431  0.667645        1.000   4.387123  0.1   
            Nonlinear1 -0.136731  0.254859        1.000   3.955234  0.1   
            Nonlinear1  -0.02318  0.505352        1.000   4.341341  0.1   
2 scenario3 Nonlinear1 -0.181618  0.552194        1.000   3.984012  0.1   
            Nonlinear1 -0.150681  0.240618        1.000   3.592942  0.1   
            Nonlinear1 -0.192379  0.470000        1.000   3.978324  0.1   
1 scenario1 Nonlinear2  0.052929  2.336547        0.950   8.186365  0.1   
            Nonlinear2  0.016747  2.028463        0.960   7.357344  0.1   
            Nonlinear2  0.062248  2.251676        0.955   8.047511  0.1   
2 scenario1 Nonlinear2 -0.267236  1.744876        0.965   6.633194  0.1   
            Nonlinear2 -0.265661  1.616926        0.960   6.131855  0.1   
            Nonlinear2 -0.270153  1.788382        0.955   6.674616  0.1   
1 scenario2 Nonlinear2  4.974949  7.380769        0.810  18.048606  0.1   
            Nonlinear2  4.865528  7.012475        0.795  16.658327  0.1   
            Nonlinear2  4.939963  7.429543        0.795  17.992443  0.1   
2 scenario2 Nonlinear2  4.647283  7.189381        0.850  17.068099  0.1   
            Nonlinear2  4.792842  6.770991        0.775   15.44743  0.1   
            Nonlinear2  4.702933  6.840538        0.785   16.75432  0.1   
1 scenario3 Nonlinear2 -5.922794  7.109124        0.695  16.326879  0.1   
            Nonlinear2 -5.793506  6.804635        0.660   14.88713  0.1   
            Nonlinear2 -5.971246  7.161557        0.700  16.067744  0.1   
2 scenario3 Nonlinear2 -4.507679  6.150922        0.7